# 13. Redes neuronales recurrentes
[**Python Deep Learning** Introducción práctica con Keras y TensorFlow 2. Jordi Torres. Editorial Marcombo ISBN: 9788426728289 ](https://www.marcombo.com/python-deep-learning-9788426728289/)

### Importar TensorFlow 2.0  y otras librerias

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

TensorFlow 2.x selected.


### Descarga del conjunto de datos

In [2]:

from google.colab import files
#se debe cargar el fichero “Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt”
files.upload()

path_to_fileDL ='/content/Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt'

#path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')


Saving Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt to Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt


In [3]:


text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        201709 carácteres
El texto está compuesto de estos 91 carácteres:
['\n', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


In [4]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  '5' :  20,
  '6' :  21,
  '7' :  22,
  '8' :  23,
  '9' :  24,
  ':' :  25,
  ';' :  26,
  '<' :  27,
  '=' :  28,
  '>' :  29,
  '?' :  30,
  '@' :  31,
  'A' :  32,
  'B' :  33,
  'C' :  34,
  'D' :  35,
  'E' :  36,
  'F' :  37,
  'G' :  38,
  'H' :  39,
  'I' :  40,
  'J' :  41,
  'K' :  42,
  'L' :  43,
  'M' :  44,
  'N' :  45,
  'O' :  46,
  'P' :  47,
  'Q' :  48,
  'R' :  49,
  'S' :  50,
  'T' :  51,
  'U' :  52,
  'V' :  53,
  'W' :  54,
  'X' :  55,
  'Y' :  56,
  '[' :  57,
  ']' :  58,
  '_' :  59,
  'a' :  60,
  'b' :  61,
  'c' :  62,
  'd' :  63,
  'e' :  64,
  'f' :  65,
  'g' :  66,
  'h' :  67,
  'i' :  68,
  'j' :  69,
  'k' :  70,
  'l' :  71,
  'm' :  72,
  'n' :  73,
  'o' :  74,
  'p' :  75,
  'q' :  76,

In [0]:
text_as_int = np.array([char2idx[c] for c in text])

In [6]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\nEn 1953, Isaac Asimov publico Segunda Fund'
array([47, 77, 74, 71, 74, 66, 74,  0, 36, 73,  1, 16, 24, 20, 18, 11,  1,
       40, 78, 60, 60, 62,  1, 32, 78, 68, 72, 74, 81,  1, 75, 80, 61, 71,
       68, 62, 74,  1, 50, 64, 66, 80, 73, 63, 60,  1, 37, 80, 73, 63])


### Preparar los datos para entrenar la RNN


In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [8]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion ('
'o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por '
'primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su prim'
'era escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccion'
' que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando u'
'n ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo, '
'que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de '
'nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de a'
'plicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Process'
'or Units), que solo cuestan alrededor de 100 euros, que estari

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
Target data: 'rologo\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion ('


In [11]:
print (dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [12]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


### Construcción del modelo

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, 
                      output_dim=embedding_dim, 
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23296     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 91)            93275     
Total params: 5,363,547
Trainable params: 5,363,547
Non-trainable params: 0
_________________________________________________________________


In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [18]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 91) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
sampled_indices_characters

array([58, 90,  3, 45, 16, 77, 39, 73, 66, 15, 86, 83, 18, 62, 45, 15, 25,
       45, 29, 29, 79, 42, 54, 59, 65, 76, 33, 62, 80, 48, 24, 15, 71, 87,
        8, 12,  3, 85, 81, 26, 48, 78, 82, 14,  0,  9, 60, 10, 68,  6, 66,
       69, 10, 34, 69, 33, 74,  4, 12, 51, 25, 75, 42, 64, 37, 57, 43, 80,
       33, 11, 24, 26, 48, 69, 48, 77, 61, 38, 90, 57, 73,  5, 22, 50, 22,
        6, 42,  3, 18, 28, 25, 24, 79, 68, 43, 66, 30, 89, 19, 48])

Entrenar el modelo

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

Configurar el *checkpoints*

In [0]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

In [24]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 31 steps
Epoch 1/50
31/31 [==============================] - 4s 118ms/step - loss: 3.2045
Epoch 2/50
31/31 [==============================] - 3s 83ms/step - loss: 2.7431
Epoch 3/50
31/31 [==============================] - 3s 84ms/step - loss: 2.4256
Epoch 4/50
31/31 [==============================] - 3s 86ms/step - loss: 2.2234
Epoch 5/50
31/31 [==============================] - 3s 85ms/step - loss: 2.1082
Epoch 6/50
31/31 [==============================] - 3s 88ms/step - loss: 1.9897
Epoch 7/50
31/31 [==============================] - 3s 86ms/step - loss: 1.8694
Epoch 8/50
31/31 [==============================] - 3s 87ms/step - loss: 1.7514
Epoch 9/50
31/31 [==============================] - 3s 91ms/step - loss: 1.6377
Epoch 10/50
31/31 [==============================] - 3s 87ms/step - loss: 1.5351
Epoch 11/50
31/31 [==============================] - 3s 86ms/step - loss: 1.4452
Epoch 12/50
31/31 [==============================] - 3s 88ms/step - loss: 1.3659
Epoch 13/50
31/31

### Generación de texto

In [25]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):

  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"a"))

arte de la primera parte del libro puede expresar de la solucion de esta manera se consigue que estamos en un espacio de neuronas. VGD en la capa de pooling. Visualmente, se puede pensar en una ventana del tamano de 5?5 que va recorriendo toda la capa de 28?28 de la capa siguiente.
Como veremos en el siguiente capitulo, hay varias funciones de activacion de la primera parte de los datos de entrenamiento y de prueba.
Si queremos comprobar que justo con el lector, programaremos una red neuronal con
